### Example of RDD toDF with both built-in functions and a UDF

use sc.parallelize() to create rdd, convert to DF and define a UDF for DF operation

In [2]:
from pyspark.sql import functions as fc
from pyspark.sql import types as tp

In [3]:
r = sc.parallelize([[1, 'Y'], [1, 'U'], [1, 'N'], [1, 'Y'], [1, 'U'], [1, 'N'],
                   [2, 'U'], [2, 'U'], [2, 'U']]).toDF(['key', 'value'])


In [4]:
r.show()

+---+-----+
|key|value|
+---+-----+
|  1|    Y|
|  1|    U|
|  1|    N|
|  1|    Y|
|  1|    U|
|  1|    N|
|  2|    U|
|  2|    U|
|  2|    U|
+---+-----+



In [5]:
r1 = r.groupBy('key').agg(fc.collect_list('value').alias('val'))

In [6]:
r1.show()

+---+------------------+
|key|               val|
+---+------------------+
|  1|[Y, U, N, Y, U, N]|
|  2|         [U, U, U]|
+---+------------------+



Below shows an example of defining a UDF to count the occurences of letter 'Y' in the lists(though this can be easily done without a UDF)

In [10]:
def num_Y(x):
    #count the occurences of 'Y' in a list
    count = 0
    for l in x:
        if l == 'Y':
            count += 1
    return count

num_Y_udf = fc.udf(num_Y, tp.IntegerType())

r2 = r.groupBy('key').agg(num_Y_udf(fc.collect_list('value')).alias('num_Y'))
    

In [11]:
r2.show()

+---+-----+
|key|num_Y|
+---+-----+
|  1|    2|
|  2|    0|
+---+-----+



Now create a dataframe with one more column

In [14]:
d = sc.parallelize([[1, 101, 'Y'], [1, 101, 'U'], [1, 102, 'N'], [1, 103, 'Y'], [1, 103, 'U'], [1, 104, 'N'],
                   [2, 201, 'U'], [2, 202, 'U'], [2, 203, 'U'], [2, 204, 'Y']]).toDF(['key', 'value1', 'value2'])

In [15]:
d.show()

+---+------+------+
|key|value1|value2|
+---+------+------+
|  1|   101|     Y|
|  1|   101|     U|
|  1|   102|     N|
|  1|   103|     Y|
|  1|   103|     U|
|  1|   104|     N|
|  2|   201|     U|
|  2|   202|     U|
|  2|   203|     U|
|  2|   204|     Y|
+---+------+------+



Below when() method is used for case of value1 == 101

In [16]:
d1 = d.groupBy('key').agg(num_Y_udf(fc.collect_list(fc.when(fc.col('value1')==101, fc.col('value2')))).alias('num_Y'))

In [17]:
d1.show()

+---+-----+
|key|num_Y|
+---+-----+
|  1|    1|
|  2|    0|
+---+-----+

